# SI 618 Fall 2020 Homework 3 (100 points)

## Part 1 (75 points) 

The provided ‘movie_actors_data.txt’ file contains a JSON string on each line. For example, the first line is: 

>{"rating": 9.3, "genres": ["Crime", "Drama"], "rated": "R", "filming_locations": "Ashland, Ohio, USA", "language": ["English"], "title": "The Shawshank Redemption", "runtime": ["142 min"], "poster": "http://img3.douban.com/lpic/s1311361.jpg", "imdb_url": "http://www.imdb.com/title/tt0111161/", "writers": ["Stephen King", "Frank Darabont"], "imdb_id": "tt0111161", "directors": ["Frank Darabont"], "rating_count": 894012, "actors": ["Tim Robbins", "Morgan Freeman", "Bob Gunton", "William Sadler", "Clancy Brown", "Gil Bellows", "Mark Rolston", "James Whitmore", "Jeffrey DeMunn", "Larry Brandenburg", "Neil Giuntoli", "Brian Libby", "David Proval", "Joseph Ragno", "Jude Ciccolella"], "plot_simple": "Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.", "year": 1994, "country": ["USA"], "type": "M", "release_date": 19941014, "also_known_as": ["Die Verurteilten"]}

The fields we are interested in are imdb_id , title , rating, genres, actors, and year. You will parse the JSON strings, and load the data into three tables in SQLite, and then write SQL queries to retrieve the data specified. 

You will create three tables: 
* The “movie_genre” table, which has two columns: imdb_id and genre. A movie typically has multiple genres, and in this case, there should be one row for each genre.**If some movie does not have any genre, ignore that movie.** 
* The “movies” table, which has four columns: imdb_id, title, year, rating, country  
* The “movie_actor” table, which has two columns imdb_id and actor. A movie typically has  multiple actors, and in this case, there should be one row for each actor. 

**When you run your Python code, it should print out EXACTLY the output provided.**

#### Q1. (10 points) Parse input file to get needed data for the three tables and load them into appropriate Python data structure. 

In [54]:
import json
import sqlite3
import os
import numpy as np
import pandas as pd

In [55]:
dbFileName = "wenyueli_si618_hw3.db"
dbFilePath = os.path.join(os.getcwd(), dbFileName)

In [56]:
conn = sqlite3.connect(dbFilePath)
c = conn.cursor()

In [57]:
lis = []
with open('movie_actors_data.txt') as f:
    for line in f.readlines():   
        dic = json.loads(line)
        lis.append(dic)
df = pd.DataFrame(lis)
df

,rating,genres,rated,filming_locations,language,title,runtime,poster,imdb_url,writers,imdb_id,directors,rating_count,actors,plot_simple,year,country,type,release_date,also_known_as
0,9.3,"[Crime, Drama]",R,"Ashland, Ohio, USA",[English],The Shawshank Redemption,[142 min],http://img3.douban.com/lpic/s1311361.jpg,http://www.imdb.com/title/tt0111161/,"[Stephen King, Frank Darabont]",tt0111161,[Frank Darabont],894012,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will...",Two imprisoned men bond over a number of years...,1994,[USA],M,19941014,[Die Verurteilten]
1,9.2,"[Crime, Drama]",R,"110 Longfellow Road, Staten Island, New York C...","[English, Italian, Latin]",The Godfather,[175 min],http://img3.douban.com/lpic/s4038344.jpg,http://www.imdb.com/title/tt0068646/,"[Mario Puzo, Francis Ford Coppola, and 1 more ...",tt0068646,[Francis Ford Coppola],646348,"[Marlon Brando, Al Pacino, James Caan, Richard...",The aging patriarch of an organized crime dyna...,1972,[USA],M,19720324,[Mario Puzo's The Godfather]
2,9.1,"[Crime, Drama]",R,"6th Street, Manhattan, New York City, New York...","[English, Italian, Spanish, Latin]",The Godfather: Part II,[200 min],http://img3.douban.com/lpic/s3314652.jpg,http://www.imdb.com/title/tt0071562/,"[Francis Ford Coppola, Mario Puzo, and 1 more ...",tt0071562,[Francis Ford Coppola],415305,"[Al Pacino, Robert Duvall, Diane Keaton, Rober...",The early life and career of Vito Corleone in ...,1974,[USA],M,19741220,[Mario Puzo's The Godfather: Part II]
3,9.0,"[Crime, Thriller]",R,"1525 N. Van Ness Avenue, Los Angeles, Californ...","[English, Spanish, French]",Pulp Fiction,"[154 min, USA: 168 min (special edition)]",http://img3.douban.com/lpic/s2845185.jpg,http://www.imdb.com/title/tt0110912/,"[Quentin Tarantino, Roger Avary, and 1 more cr...",tt0110912,[Quentin Tarantino],694866,"[John Travolta, Samuel L. Jackson, Tim Roth, A...","The lives of two mob hit men, a boxer, a gangs...",1994,[USA],M,19941014,[Tiempos violentos]
4,9.0,"[Adventure, Western]",APPROVED,"Almería, Andalucía, Spain",[Italian],"Il buono, il brutto, il cattivo.","[161 min, France: 186 min (dubbed version), Sp...",http://img3.douban.com/lpic/s4008151.jpg,http://www.imdb.com/title/tt0060196/,"[Luciano Vincenzoni, Sergio Leone]",tt0060196,[Sergio Leone],274781,"[Eli Wallach, Clint Eastwood, Lee Van Cleef, A...",A bounty hunting scam joins two men in an unea...,1966,"[Italy, Spain, West Germany]",M,19671229,"[The Good, the Bad and the Ugly]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,8.1,"[Animation, Action, Adventure, Family, Fantasy...",PG,NaN,[Japanese],Tenkû no shiro Rapyuta,[124 min],http://ia.media-imdb.com/images/M/MV5BMTU4MTUy...,http://www.imdb.com/title/tt0092067/,[Hayao Miyazaki],tt0092067,[Hayao Miyazaki],43226,"[Barbara Goodson, James Van Der Beek, Lara Cod...",A young boy and a girl with a magic crystal mu...,1986,[Japan],M,19860802,[Castle in the Sky]
244,8.0,"[Crime, Drama, History, Thriller]",R,"40 miles south, Great Falls, Montana, USA",[English],The Untouchables,[119 min],http://img3.douban.com/lpic/s1429054.jpg,http://www.imdb.com/title/tt0094226/,"[Oscar Fraley, Eliot Ness, and 1 more credit]",tt0094226,[Brian De Palma],138472,"[Kevin Costner, Sean Connery, Charles Martin S...",Federal Agent Eliot Ness sets out to stop Al C...,1987,[USA],M,19870603,[Los intocables]
245,8.0,[Thriller],TV_PG,"904 McDonald Avenue, Santa Rosa, California, USA",[English],Shadow of a Doubt,[108 min],http://img3.douban.com/lpic/s2874106.jpg,http://www.imdb.com/title/tt0036342/,"[Thornton Wilder, Sally Benson]",tt0036342,[Alfred Hitchcock],30904,"[Teresa Wright, Joseph Cotten, Macdonald Carey...","A young woman discovers her visiting ""Uncle Ch...",1943,[USA],M,19430115,[Shadow of Doubt]
246,8.0,"[Comedy, Crime, Romance, Thriller]",NOT_RATED,"City Island, Bronx, New York City, New York, USA","[English, German]",Arsenic and Old Lace,[118 min],http://img3.douban.com/lpic/s3015765.jpg,http://www.imdb.com/title/

In [58]:
df.isnull().any()

rating               False
genres               False
rated                 True
filming_locations     True
language              True
title                False
runtime              False
poster               False
imdb_url             False
writers              False
imdb_id              False
directors            False
rating_count         False
actors               False
plot_simple          False
year                 False
country              False
type                 False
release_date         False
also_known_as        False
dtype: bool

In [59]:
def render(query):
    result = pd.DataFrame(c.execute(query))
    return result

#### Q2. (5 points) Create the movie_genre table and load data into it.

In [60]:
movie_genre_df = df.loc[:,['imdb_id','genres']].explode('genres')
movie_genre = list(movie_genre_df.to_records(index=False))
#movie_genre

In [61]:
c.execute('DROP TABLE movie_genre')

In [62]:
c.execute('CREATE TABLE movie_genre (imdb_id  text  not null, genre  text  not null)')
c.executemany('INSERT INTO movie_genre VALUES(?,?)',movie_genre)
conn.commit()
#print("insert",movie_genre,"success")

#### Q3. (5 points) Create the movies table and load data into it.

movie_df = df.loc[:,['imdb_id', 'title', 'year', 'rating', 'country']].explode('country')
movies = list(movie_df.to_records(index=False))

c.execute('DROP TABLE movie')

c.execute('CREATE TABLE movie (imdb_id text, title text, year real, rating float, country text)')
c.executemany('INSERT INTO movie VALUES(?,?,?,?,?)', movies)
conn.commit()
print("insert",movies,"success")

In [63]:
c.execute('DROP TABLE movies')

In [64]:
c.execute('CREATE TABLE movies (imdb_id text, title text, year int, rating float, country text)')
with open('movie_actors_data.txt') as f:
    for line in f.readlines():
        dic = json.loads(line)
        imdb_id = dic['imdb_id']
        title = dic['title']
        rating = dic['rating']
        year = dic['year']
        for country in dic['country']:
            data = (imdb_id, title, year, rating, country)
            c.execute('INSERT INTO movies VALUES(?,?,?,?,?)', data)
            conn.commit()
            #print('insert'+ str(data)+'success')

#### Q4. (5 points) Create the movie_actor table and load data into it.

In [65]:
movie_actor_df = df.loc[:,['imdb_id','actors']].explode('actors')
movie_actor = list(movie_actor_df.to_records(index=False))

In [66]:
c.execute('DROP TABLE movie_actor')

In [1]:
c.execute('CREATE TABLE movie_actor (imdb_id text,actor text)')
c.executemany('INSERT INTO movie_actor VALUES(?,?)',movie_actor)
conn.commit()
print("insert",movie_actor,"success")

NameError: name 'c' is not defined

#### Q5. (10 points) Write a SQL query to find top 10 actors starring in the most number of movies and print out the results.
```
Top 10 actors:Actor, Movies
Robert De Niro,10
James Stewart,8
Morgan Freeman,7
Harrison Ford,7
Michael Caine,6
John Ratzenberger,6
Clint Eastwood,6
William Holden,5
Tom Hanks,5
Steve Buscemi,5
```

In [68]:
render('SELECT actor, count(actor) FROM movie_actor GROUP BY actor ORDER BY 2 DESC LIMIT 10')

,0,1
0,Robert De Niro,10
1,James Stewart,8
2,Morgan Freeman,7
3,Harrison Ford,7
4,Michael Caine,6
5,John Ratzenberger,6
6,Clint Eastwood,6
7,William Holden,5
8,Tom Hanks,5
9,Steve Buscemi,5


#### Q6. (10 points) Write a SQL query to find the number of movies broken down by year in chronological order.
```
Movies broken down by year:Year, Movies
1921, 1
1922, 1
1925, 1
1926, 1
1927, 1
1930, 1
1931, 2
1934, 1
1936, 1
1939, 3
1940, 3
1941, 2
1942, 1
1943, 1
1944, 2
1946, 3
1948, 3
1949, 1
1950, 4
1951, 2
1952, 2
1953, 2
1954, 5
1955, 1
1956, 1
1957, 6
1958, 2
1959, 5
1960, 2
1961, 2
1962, 4
1963, 2
1964, 2
1965, 1
1966, 3
1967, 2
1968, 3
1969, 2
1971, 1
1972, 2
1973, 3
1974, 2
1975, 5
1976, 3
1977, 2
1978, 1
1979, 5
1980, 3
1981, 2
1982, 3
1983, 1
1984, 3
1985, 2
1986, 3
1987, 3
1988, 5
1989, 1
1990, 1
1991, 3
1992, 2
1993, 4
1994, 5
1995, 8
1996, 2
1997, 3
1998, 5
1999, 5
2000, 5
2001, 5
2002, 5
2003, 9
2004, 5
2005, 3
2006, 4
2007, 7
2008, 7
2009, 7
2010, 7
2011, 6
2012, 8
```

(1922).to_bytes(8, byteorder="little")

b = b'\x82\x07\x00\x00\x00\x00\x00\x00'
int.from_bytes(b, 'little')

a = c.execute('SELECT year, count(distinct title) FROM movie GROUP BY year ORDER BY 1')
for i in a:
    b = int.from_bytes(i[0], 'little')
    print('{}, {}'.format(b,i[1]))

In [69]:
render('SELECT year, count(distinct title) FROM movies GROUP BY year ORDER BY 1')

,0,1
0,1921,1
1,1922,1
2,1925,1
3,1926,1
4,1927,1
...,...,...
75,2008,7
76,2009,7
77,2010,7
78,2011,6


#### Q7. (10 points) Write a SQL query to find all Mystery movies from the U.S. ordered by increasing rating, then by decreasing year if ratings are the same.

```
Mystery movies:Title, Year, Rating
Shutter Island, 2010, 8.0
Mystic River, 2003, 8.0
Rosemary's Baby, 1968, 8.0
Harry Potter and the Deathly Hallows: Part 2, 2011, 8.1
Black Swan, 2010, 8.1
Twelve Monkeys, 1995, 8.1
Sleuth, 1972, 8.1
The Manchurian Candidate, 1962, 8.1
Anatomy of a Murder, 1959, 8.1
Rope, 1948, 8.1
Donnie Darko, 2001, 8.2
The Sixth Sense, 1999, 8.2
The Thing, 1982, 8.2
Dial M for Murder, 1954, 8.2
Strangers on a Train, 1951, 8.2
The Big Sleep, 1946, 8.2
The Maltese Falcon, 1941, 8.3
Rebecca, 1940, 8.3
L.A. Confidential, 1997, 8.4
Chinatown, 1974, 8.4
2001: A Space Odyssey, 1968, 8.4
Witness for the Prosecution, 1957, 8.4
The Prestige, 2006, 8.5
The Green Mile, 1999, 8.5
To Kill a Mockingbird, 1962, 8.5
Vertigo, 1958, 8.5
Citizen Kane, 1941, 8.5
Memento, 2000, 8.6
Psycho, 1960, 8.6
Se7en, 1995, 8.7
The Usual Suspects, 1995, 8.7
Rear Window, 1954, 8.7
Inception, 2010, 8.8
12 Angry Men, 1957, 8.9
```

In [70]:
render('''SELECT distinct title, movies.year, movies.rating 
FROM movies INNER JOIN movie_genre ON (movies.imdb_id = movie_genre.imdb_id) 
WHERE movie_genre.genre LIKE 'Mystery' AND movies.country LIKE 'USA' ORDER BY 3 ASC, 2 DESC''')

,0,1,2
0,Shutter Island,2010,8.0
1,Mystic River,2003,8.0
2,Rosemary's Baby,1968,8.0
3,Harry Potter and the Deathly Hallows: Part 2,2011,8.1
4,Black Swan,2010,8.1
5,Twelve Monkeys,1995,8.1
6,Sleuth,1972,8.1
7,The Manchurian Candidate,1962,8.1
8,Anatomy of a Murder,1959,8.1
9,Rope,1948,8.1


d = c.execute('''SELECT distinct title, movie.year, movie.rating 
FROM movie INNER JOIN movie_genre ON (movie.imdb_id = movie_genre.imdb_id) 
WHERE movie_genre.genre LIKE 'Mystery' AND movie.country LIKE 'USA' ORDER BY 3 ASC, 2 DESC''')
for item in d.fetchall():
    e = int.from_bytes(item[1], 'little')
    print("{}, {}, {}".format(item[0],e,item[2]))

#### Q8. (10 points) Write a SQL query to find the top 10 actors that played in the highest rated movies (on average) after year 2000. 
For each actor, give their name, average rating of the movies they played in, and the number of movies. Sort the result from descending order based on average movie rating. In case of ties, sort the rows by actor name.

```
In and after year 2000, top 10 actors who played in the highest rated movies: Actor, Movies
Aaron Eckhart, 1, 9.0
Anthony Michael Hall, 1, 9.0
Chin Han, 1, 9.0
Eric Roberts, 1, 9.0
Heath Ledger, 1, 9.0
Monique Gabriela Curnen, 1, 9.0
Nestor Carbonell, 1, 9.0
Ritchie Coster, 1, 9.0
Ron Dean, 1, 9.0
Alexandra Astin, 1, 8.9
```


In [71]:
render('''SELECT movie_actor.actor, count(distinct movies.title), AVG(movies.rating) 
FROM movie_actor INNER JOIN movies ON (movie_actor.imdb_id = movies.imdb_id) 
WHERE movies.year > 2000
GROUP BY movie_actor.actor ORDER BY 3 DESC, 1 ASC LIMIT 10''')

,0,1,2
0,Aaron Eckhart,1,9.0
1,Anthony Michael Hall,1,9.0
2,Chin Han,1,9.0
3,Eric Roberts,1,9.0
4,Heath Ledger,1,9.0
5,Monique Gabriela Curnen,1,9.0
6,Nestor Carbonell,1,9.0
7,Ritchie Coster,1,9.0
8,Ron Dean,1,9.0
9,Alexandra Astin,1,8.9


#### Q9. (10 points) Write a SQL query for finding pairs of actors who co-starred in at least 3 highly rated (rating > 8.5) movies together. 

The pairs of names must be unique. This means that ‘actor A, actor B’ and ‘actor B, actor A’ are the same pair, so only one of them should appear. For each pair of actors you print out, the two actors must be ordered alphabetically. The pairs are ordered in decreasing number of movies they co-stared in. In case of ties, the rows are ordered by actors’ names. 

>You will need to join the movie_actor table with itself to get this data (in addition to another required join). It is a bit tricky. If you cannot do it with SQL statement, you can also write some Python code that works on the Python data structure that you used to create the movie_actor table. That’ll mean much more lines of code, and if you do it that way, you’ll get 5 points instead of 10 points. You will only get 10 points if you solve it with pure SQL.

```
Pairs of actors who co-stared in 3 or more highly rated movies: Actor A, Actor B, Co-stared Movies
Billy Boyd, Cate Blanchett, 3
Billy Boyd, Orlando Bloom, 3
Billy Boyd, Sean Astin, 3
Cate Blanchett, Orlando Bloom, 3
Cate Blanchett, Sean Astin, 3
Orlando Bloom, Sean Astin, 3
```

In [72]:
render('''SELECT movie_actor.actor, actors.actor, count(movies.imdb_id)/2 
FROM movie_actor JOIN movie_actor AS actors ON movie_actor.imdb_id = actors.imdb_id JOIN movies ON movie_actor.imdb_id = movies.imdb_id
WHERE movie_actor.actor < actors.actor AND movies.rating > 8.5 
GROUP BY 1,2
HAVING count(movies.imdb_id) >=3 
ORDER BY count(movies.imdb_id) DESC, movie_actor.actor ASC, actors.actor ASC
LIMIT 6
''')

,0,1,2
0,Billy Boyd,Cate Blanchett,3
1,Billy Boyd,Orlando Bloom,3
2,Billy Boyd,Sean Astin,3
3,Cate Blanchett,Orlando Bloom,3
4,Cate Blanchett,Sean Astin,3
5,Orlando Bloom,Sean Astin,3


## Part 2 (25 points) 

Write a python function that takes two arguments: genre and k 

The function should print out the top k actors who played roles in the highest rated movies (on average) in the provided genre. You should use the sqlite3 database file you created in Part 1. Some example runs of my function are shown below. Your program should produce the same output when provided with the same command line arguments.

`get_top_genre_actors("Drama", 5)`
```
Top 5 actors who played in highest rated Drama movies:
Actor, Average Drama movie rating
Bob Gunton, 9.3
Brian Libby, 9.3
Clancy Brown, 9.3
David Proval, 9.3
Gil Bellows, 9.3
```

`get_top_genre_actors("Action", 10)`
```
Top 10 actors who played in highest rated Action movies:
Actor, Average Action movie rating
Ron Dean, 9.0
Ritchie Coster, 9.0
Nestor Carbonell, 9.0
Monique Gabriela Curnen, 9.0
Maggie Gyllenhaal, 9.0
Heath Ledger, 9.0
Eric Roberts, 9.0
Chin Han, 9.0
Anthony Michael Hall, 9.0
Aaron Eckhart, 9.0
```


`get_top_genre_actors("Comedy", 10)`
```
Top 10 actors who played in highest rated Comedy movies:
Actor, Average Comedy movie rating
Wilfred Lucas, 8.6
Virginia Cherrill, 8.6
Tracy Reed, 8.6
Tiny Sandford, 8.6
Thomas Solivéres, 8.6
Sterling Hayden, 8.6
Stanley Blystone, 8.6
Slim Pickens, 8.6
Shane Rimmer, 8.6
Roy Stephens, 8.6
```

In [73]:
def get_top_genre_actors(genre:str, k:int):
    top = c.execute('''SELECT actor, AVG(rating) FROM movies JOIN movie_genre ON movies.imdb_id = movie_genre.imdb_id 
                    JOIN movie_actor ON movies.imdb_id = movie_actor.imdb_id 
                    WHERE movie_genre.genre = ? GROUP BY movie_actor.actor ORDER BY 2 DESC LIMIT ?''', (genre, k))
    for i in top:
        print("{}, {}".format(i[0],i[1]))

In [74]:
get_top_genre_actors("Drama", 5)

William Sadler, 9.3
Neil Giuntoli, 9.3
Mark Rolston, 9.3
Larry Brandenburg, 9.3
Joseph Ragno, 9.3


In [75]:
get_top_genre_actors("Action", 10)

Ron Dean, 9.0
Ritchie Coster, 9.0
Nestor Carbonell, 9.0
Monique Gabriela Curnen, 9.0
Maggie Gyllenhaal, 9.0
Heath Ledger, 9.0
Eric Roberts, 9.0
Chin Han, 9.0
Anthony Michael Hall, 9.0
Aaron Eckhart, 9.0


In [76]:
get_top_genre_actors("Comedy", 10)

Wilfred Lucas, 8.6
Virginia Cherrill, 8.6
Tracy Reed, 8.6
Tiny Sandford, 8.6
Thomas Solivéres, 8.6
Sterling Hayden, 8.6
Stanley Blystone, 8.6
Slim Pickens, 8.6
Shane Rimmer, 8.6
Roy Stephens, 8.6


## What to submit: 
Please perform the following operations and turn in the corresponding files to canvas:
1. Jupyter notebook titled uniqname_si618_hw3.ipynb and the corresponding html page (uniqname_si618_hw3.html) 
through Canvas.
2. The sqlite database file you created. (uniqname_si618_hw3.db)